# Denoising with generative models

## Pytorch tests

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [11]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [12]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1470,  0.1537, -0.0096,  0.0258, -0.0937,  0.0925, -0.0244, -0.0572,
         -0.1353, -0.1321]], grad_fn=<AddmmBackward>)


In [13]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.8868, grad_fn=<MseLossBackward>)


In [14]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-5.7337e-03,  1.0262e-02,  9.8151e-03,  6.4877e-05, -1.4198e-03,
        -6.3296e-03])


In [16]:
net.conv1.weight

Parameter containing:
tensor([[[[-0.2095, -0.3291, -0.1576],
          [-0.1975,  0.1326,  0.0288],
          [ 0.1704,  0.2602, -0.0633]]],


        [[[ 0.0624, -0.1205,  0.1666],
          [ 0.3153,  0.2187, -0.1577],
          [ 0.0573,  0.1785, -0.0375]]],


        [[[-0.1807, -0.0485, -0.1532],
          [ 0.2872, -0.0977,  0.1736],
          [-0.2958, -0.1144,  0.0846]]],


        [[[-0.0628, -0.0436, -0.1161],
          [-0.1117, -0.1268,  0.3123],
          [-0.2840,  0.1628,  0.2471]]],


        [[[ 0.2705,  0.0535,  0.3081],
          [-0.1068,  0.3209, -0.0607],
          [-0.2062,  0.3302, -0.0711]]],


        [[[-0.1150, -0.1316, -0.2298],
          [ 0.2041, -0.1024, -0.1937],
          [-0.3270,  0.2065,  0.1366]]]], requires_grad=True)

In [17]:
net.conv1.bias

Parameter containing:
tensor([ 0.1650,  0.2838, -0.0661, -0.2212, -0.1095, -0.0561],
       requires_grad=True)